In [27]:
import datasets

from argparse import Namespace
from collections import namedtuple
from fairseq.data.dictionary import Dictionary
from fairseq.data.encoders.gpt2_bpe import GPT2BPE

Cfg = namedtuple("Cfg", ["gpt2_encoder_json", "gpt2_vocab_bpe"])
cfg = Cfg("../artifacts/gpt_icl/encoder.json", "../artifacts/gpt_icl/vocab.bpe")
tokenizer =  GPT2BPE(
    cfg=cfg,
)
dictionary = Dictionary.load("../artifacts/gpt_icl/en_dense_lm_1_3b/dict.txt")
text_cols = {'premise', 'hypothesis', 'text', 'sentence'}
tasks = ['sst2', ('super_glue', 'cb'), 'SetFit/sst5', 'rotten_tomatoes', 'SetFit/subj']
lengths = []
sample_sizes = []
for task in tasks:
    dataset = datasets.load_dataset(task if isinstance(task, str) else task[0], task[1] if isinstance(task, tuple) else None)
    text_col = list(text_cols.intersection(set(dataset['train'].column_names)))[0]
    dataset_l = dataset['train'].map(lambda d: {'l': len(tokenizer.encode(d[text_col]))})
    avg_toks = sum(list(dataset_l['l'])) / len(dataset['train'])
    lengths.append(len(dataset['train'])) # print(task, "length", len(dataset['train']))
    sample_sizes.append(avg_toks) # print(task, "avg_toks per sample", avg_toks)

for task in tasks:
    print("------------------", task, "------------------")
    print(task, "length", lengths[tasks.index(task)])
    print(task, "avg_toks per sample", sample_sizes[tasks.index(task)])

2023-12-12 23:58:15 | WARNING | datasets.builder | Using custom data configuration default
2023-12-12 23:58:15 | WARNING | datasets.builder | Reusing dataset sst2 (/home/joberant/NLP_2223/nadavmagar/.cache/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
100%|██████████| 3/3 [00:00<00:00, 486.48it/s]
2023-12-12 23:58:16 | WARNING | datasets.arrow_dataset | Loading cached processed dataset at /home/joberant/NLP_2223/nadavmagar/.cache/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-e02e028727bd4063.arrow
2023-12-12 23:58:18 | WARNING | datasets.builder | Reusing dataset super_glue (/home/joberant/NLP_2223/nadavmagar/.cache/super_glue/cb/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)
100%|██████████| 3/3 [00:00<00:00, 550.58it/s]
2023-12-12 23:58:19 | WARNING | datasets.arrow_dataset | Loading cached processed dataset at /home/joberant/NLP_2223/nadavmagar/.cache/super_glue/cb/1.0.2/d040c

------------------ sst2 ------------------
sst2 length 67349
sst2 avg_toks per sample 55.42990987245542
------------------ ('super_glue', 'cb') ------------------
('super_glue', 'cb') length 250
('super_glue', 'cb') avg_toks per sample 295.804
------------------ SetFit/sst5 ------------------
SetFit/sst5 length 8544
SetFit/sst5 avg_toks per sample 102.9492041198502
------------------ rotten_tomatoes ------------------
rotten_tomatoes length 8530
rotten_tomatoes avg_toks per sample 113.3852286049238
------------------ SetFit/subj ------------------
SetFit/subj length 8000
SetFit/subj avg_toks per sample 129.23025
